In [1]:
from Simulation_PyAPDL import simulation_PyAPDL
from scipy.integrate import simps
import matplotlib.pyplot as plt
import scipy.io as sio
import numpy as np
import json
import os
from IPython.display import clear_output


simu_tk = simulation_PyAPDL(nproc=4,nerr=10000)
simu_tk.launch_engine()
simu_tk.mapdl.clear()

Launch Pymapdl
     Launch Pymapdl successfully, duration 5.2164 seconds.


### EX 220

In [ ]:

DOE_info = {}
vary_ratio = 0.5
# Load the JSON file
with open('lhs_data.json', 'r') as json_file:
    loaded_data = json.load(json_file)

# Convert back to a NumPy array if needed
X = np.array(loaded_data)
directory = r"D:/MDSI_project/PCE_surrogate/InputData/Realization_frf_ex220/"

# Transform X from [0,1] to [-1,1]
X_transformed = 2 * X - 1

# 154 - 159 usgin 18 modeal
# 
for i_doe in range(0,400):
    filename = f"DOE_result_{i_doe}.json"
    full_path = os.path.join(directory, filename)
    X_doe = X[i_doe,:]
    X_doe_transformed = X_transformed[i_doe,:]

    
    simu_tk.mapdl.clear()

    # Parameter list    = [slab ,corridor, ground, edge_wall, inner_wall, stair ]
    # Parameter list    = [ vs,   rho,   nu ]
    t_para   = np.array([ 0.468+ vary_ratio*(0.468)*X_transformed[i_doe,0],
                          0.8  + vary_ratio*(0.8  )*X_transformed[i_doe,1],
                          0.05 + vary_ratio*(0.05 )*X_transformed[i_doe,2],
                          0.2  + vary_ratio*(0.2  )*X_transformed[i_doe,3],
                          0.2  + vary_ratio*(0.2  )*X_transformed[i_doe,4],
                          0.468+ vary_ratio*(0.468)*X_transformed[i_doe,0]])
    
    e_para   = np.array([ 20e9 + vary_ratio*20e9*X_transformed[i_doe,5],
                          20e9 + vary_ratio*20e9*X_transformed[i_doe,6],
                          20e9 + vary_ratio*20e9*X_transformed[i_doe,7],
                          20e9 + vary_ratio*20e9*X_transformed[i_doe,8],
                          20e9 + vary_ratio*20e9*X_transformed[i_doe,9],
                          20e9 + vary_ratio*20e9*X_transformed[i_doe,5]])
    
    nu_para  = np.array([ 0.27,
                          0.27,
                          0.27,
                          0.27,
                          0.27,
                          0.27])
    
    rho_para = np.array([ 2300 + vary_ratio*2300*X_transformed[i_doe,10],
                          2300 + vary_ratio*2300*X_transformed[i_doe,11],
                          2300 + vary_ratio*2300*X_transformed[i_doe,12],
                          2300 + vary_ratio*2300*X_transformed[i_doe,13],
                          2300 + vary_ratio*2300*X_transformed[i_doe,14],
                          2300 + vary_ratio*2300*X_transformed[i_doe,10]])

    soil_para = np.array([ 350  + vary_ratio*(350 )*X_transformed[i_doe,15],
                           1400 + vary_ratio*(1400)*X_transformed[i_doe,16],
                           0.33 ])
    
    height_b            = 3.37 + vary_ratio*3.37*X_transformed[i_doe,17]
    damp_ratio_b        = 0.045+ 0.01*X_transformed[i_doe,18]

    simu_tk.setting_parameter(bool_SSI= True,height=height_b, t_arr=t_para, e_arr=e_para, nu_arr=nu_para, rho_arr=rho_para, soil_arr=soil_para)
    simu_tk.build_model()


    # Excitation point: Hammer 2.0 Pos1 (shaker)
    tol = 0.3
    simu_tk.mapdl.nsel('S','LOC','Z',simu_tk.height*2)
    simu_tk.mapdl.nsel('R','LOC','X',7.19-tol,7.19+tol)
    nsel_id_2 = simu_tk.mapdl.nsel('R','LOC','Y',8.14-tol,8.14+tol)
    simu_tk.mapdl.nsel('ALL')
    
    # Apply the force on selected node and solve 
    simu_tk.solve_model(Solu_type=6, N_modes = 40, Freq_Incr=1, End_Freq=50,Damping_ratio= damp_ratio_b, excitation_node=nsel_id_2[1])
    

    modal_info = simu_tk.get_all_modal_info(N_modes_exp=40)

 
    ## Get FRF at target point
    meas_FRFs = simu_tk.get_FRFs_meas_EXAMPLE()
  
    
    DOE_info = {
        'id':i_doe,
        'X':X_doe.tolist(),
        'X_trans':X_doe_transformed.tolist(),
        't_info':simu_tk.t_arr.tolist(),
        'e_info':simu_tk.e_arr.tolist(),
        'n_info':simu_tk.nu_arr.tolist(),
        'r_info':simu_tk.rho_arr.tolist(),
        's_info':simu_tk.soil_arr.tolist(),
        'height':simu_tk.height,
        'dr':simu_tk.Damping_ratio,
        'modal_info':modal_info,
        'meas_frf':meas_FRFs
        }
    with open(full_path, "w") as file:
        json.dump(DOE_info, file, indent=4)  # `indent=4` for pretty printing
    simu_tk.mapdl.clear()
    clear_output(wait=True)

 Setting information
     Building inner wall           :  True
     Building stair                :  True
     Building SSI                  :  True
     Element size                  :  0.4
     Height of building            :  3.750319335440995
 Assigning parameter
     Rewrite thickness by input
     Rewrite Young's modulus by input
     Rewrite Poisson's ratio by input
     Rewrite Density by input
     Rewrite soil properties by input
 /PREP7 activated
     /PREP7 finished, duration 6.6718 seconds.
Solve model: /SOLU
 Setting information
     Solution type                 :  6
     Start frequency               :  0.001
     End frequency                 :  50
     Freqency increment            :  1
     Number of modes (solu_type=2) :  40
     Damping ratio                 :  0.050753274340315006
     Excitation node               :  5344
Using Modal-superpostion method
     /SOLU finished, duration 30.8689 seconds.
 Extract all modal information , duration 4.0511 seconds.
 /POS

### EX 221

In [ ]:
import os
from IPython.display import clear_output

DOE_info = {}
vary_ratio = 0.5
# Load the JSON file
with open('lhs_data.json', 'r') as json_file:
    loaded_data = json.load(json_file)

# Convert back to a NumPy array if needed
X = np.array(loaded_data)
directory = r"D:/MDSI_project/PCE_surrogate/InputData/Realization_frf_ex221/"

# Transform X from [0,1] to [-1,1]
X_transformed = 2 * X - 1

# 154 - 159 usgin 18 modeal
# 
for i_doe in range(0,400):
    filename = f"DOE_result_{i_doe}.json"
    full_path = os.path.join(directory, filename)
    X_doe = X[i_doe,:]
    X_doe_transformed = X_transformed[i_doe,:]

    
    simu_tk.mapdl.clear()

    # Parameter list    = [slab ,corridor, ground, edge_wall, inner_wall, stair ]
    # Parameter list    = [ vs,   rho,   nu ]
    t_para   = np.array([ 0.468+ vary_ratio*(0.468)*X_transformed[i_doe,0],
                          0.8  + vary_ratio*(0.8  )*X_transformed[i_doe,1],
                          0.05 + vary_ratio*(0.05 )*X_transformed[i_doe,2],
                          0.2  + vary_ratio*(0.2  )*X_transformed[i_doe,3],
                          0.2  + vary_ratio*(0.2  )*X_transformed[i_doe,4],
                          0.468+ vary_ratio*(0.468)*X_transformed[i_doe,0]])
    
    e_para   = np.array([ 20e9 + vary_ratio*20e9*X_transformed[i_doe,5],
                          20e9 + vary_ratio*20e9*X_transformed[i_doe,6],
                          20e9 + vary_ratio*20e9*X_transformed[i_doe,7],
                          20e9 + vary_ratio*20e9*X_transformed[i_doe,8],
                          20e9 + vary_ratio*20e9*X_transformed[i_doe,9],
                          20e9 + vary_ratio*20e9*X_transformed[i_doe,5]])
    
    nu_para  = np.array([ 0.27,
                          0.27,
                          0.27,
                          0.27,
                          0.27,
                          0.27])
    
    rho_para = np.array([ 2300 + vary_ratio*2300*X_transformed[i_doe,10],
                          2300 + vary_ratio*2300*X_transformed[i_doe,11],
                          2300 + vary_ratio*2300*X_transformed[i_doe,12],
                          2300 + vary_ratio*2300*X_transformed[i_doe,13],
                          2300 + vary_ratio*2300*X_transformed[i_doe,14],
                          2300 + vary_ratio*2300*X_transformed[i_doe,10]])

    soil_para = np.array([ 350  + vary_ratio*(350 )*X_transformed[i_doe,15],
                           1400 + vary_ratio*(1400)*X_transformed[i_doe,16],
                           0.33 ])
    
    height_b            = 3.37 + vary_ratio*3.37*X_transformed[i_doe,17]
    damp_ratio_b        = 0.045+ 0.01*X_transformed[i_doe,18]

    simu_tk.setting_parameter(bool_SSI= True,height=height_b, t_arr=t_para, e_arr=e_para, nu_arr=nu_para, rho_arr=rho_para, soil_arr=soil_para)
    simu_tk.build_model()


    tol = 0.3
    # Excitation point: Hammer 2.1 Pos1 (close to edge)
    simu_tk.mapdl.nsel('S','LOC','Z',simu_tk.height*2)
    simu_tk.mapdl.nsel('R','LOC','X',12.776-tol,12.776+tol)
    nsel_id_3 = simu_tk.mapdl.nsel('R','LOC','Y',2.748-tol,2.748+tol)
    simu_tk.mapdl.nsel('ALL')
    # Apply the force on selected node and solve 
    simu_tk.solve_model(Solu_type=6, N_modes = 40, Freq_Incr=1, End_Freq=50,Damping_ratio= damp_ratio_b, excitation_node=nsel_id_3[1])
    modal_info = simu_tk.get_all_modal_info(N_modes_exp=40)
    

    modal_info = simu_tk.get_all_modal_info(N_modes_exp=40)

 
    ## Get FRF at target point
    meas_FRFs = simu_tk.get_FRFs_meas_EXAMPLE()
  
    
    DOE_info = {
        'id':i_doe,
        'X':X_doe.tolist(),
        'X_trans':X_doe_transformed.tolist(),
        't_info':simu_tk.t_arr.tolist(),
        'e_info':simu_tk.e_arr.tolist(),
        'n_info':simu_tk.nu_arr.tolist(),
        'r_info':simu_tk.rho_arr.tolist(),
        's_info':simu_tk.soil_arr.tolist(),
        'height':simu_tk.height,
        'dr':simu_tk.Damping_ratio,
        'modal_info':modal_info,
        'meas_frf':meas_FRFs
        }
    with open(full_path, "w") as file:
        json.dump(DOE_info, file, indent=4)  # `indent=4` for pretty printing
    simu_tk.mapdl.clear()
    clear_output(wait=True)

 Setting information
     Building inner wall           :  True
     Building stair                :  True
     Building SSI                  :  True
     Element size                  :  0.4
     Height of building            :  2.958379359684112
 Assigning parameter
     Rewrite thickness by input
     Rewrite Young's modulus by input
     Rewrite Poisson's ratio by input
     Rewrite Density by input
     Rewrite soil properties by input
 /PREP7 activated
     /PREP7 finished, duration 6.7694 seconds.
Solve model: /SOLU
 Setting information
     Solution type                 :  6
     Start frequency               :  0.001
     End frequency                 :  50
     Freqency increment            :  1
     Number of modes (solu_type=2) :  40
     Damping ratio                 :  0.03633969249919865
     Excitation node               :  5757
Using Modal-superpostion method
     /SOLU finished, duration 36.1143 seconds.
 Extract all modal information , duration 5.1128 seconds.
 Extra